In [1]:
import csv

def SaveOnFile(FileName, title, location, company,salary, sponsored, ratings,DatePost,job_desc, remarks):
    with open(FileName + '.csv', 'a',newline='') as jobsearch_file:
        jobsearch_file = csv.writer(jobsearch_file)
        jobsearch_file.writerow([title, location, company,salary, sponsored, ratings,DatePost,job_desc, remarks])

def SaveOnFileFails(FileName, title, location, company, RowFail):
    with open(FileName + '.csv', 'a',newline='') as jobsearch_file:
        jobsearch_file = csv.writer(jobsearch_file)
        jobsearch_file.writerow([title, location, company,RowFail])

        
#, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL


In [6]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from bs4 import BeautifulSoup
import time

FastRun = 0   #(0 or 1)
if FastRun == 1:
    sleeptime = 1
else:
    sleeptime = 4

driver = webdriver.Chrome('./chromedriver')
def RemarksValue (RemarkIndex, Type):
    try:
        if Type == 1:
            return driver.find_element_by_xpath('//*[@id="vjs-tab-job"]/div[1]/div['+str(RemarkIndex)+']/span[2]').text
        else:
            return driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div/div/div[1]/div[1]/div[3]/div[1]/div['+str(RemarkIndex)+']/span').text
    except:
        return 0

dataframe = pd.DataFrame(columns=['Title','Location','Company','Salary','Sponsored','Ratings','DatePost','Description', 'Remarks'])
fails = pd.DataFrame(columns=['Title','Location','Company','RowFail'])

DataFileName = 'Data'
FailFileName = 'Fails'

dataframe.to_csv(DataFileName + '.csv',index=False)
dataframe.to_csv(FailFileName + '.csv',index=False)


Extension = ''
for i in range(0,600,1):
    
    if i >= 1:
        Extension = '&filter=0'    
    
    for y in range(0,1000,10):
        
        driver.get('https://ca.indeed.com/jobs?q=data+science&l=Canada'+str(Extension)+'&start='+str(y))
        driver.implicitly_wait(4)
        #act = ActionChains(driver)
        #act.send_keys(Keys.END).perform()
        
        all_jobs=driver.find_elements_by_class_name('result')
        
        for job in all_jobs:
            try:
                result_html = job.get_attribute('innerHTML')
                soup = BeautifulSoup(result_html,'html.parser')
            except:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(sleeptime)
                result_html = job.get_attribute('innerHTML')
                soup = BeautifulSoup(result_html,'html.parser')                    
            try:
                title = soup.find('a', class_='jobtitle').text.replace('\n','')
            except:
                title = None
            try:
                location = soup.find(class_='location').text
            except:
                location = None
            try:
                company = soup.find(class_='company').text.replace('\n','').strip()
            except:
                company = None
            try:
                salary = soup.find(class_='salary').text.replace('\n','').strip()
            except:
                salary = None
            try:
                sponsored = soup.find(class_='sponsoredGray').text
            except:
                sponsored = 'Organic'
            try:
                ratings = soup.find(class_='ratingsContent').text.replace('\n','')
            except:
                ratings = None
            try:
                DatePost = soup.find(class_='date').text
            except:
                DatePost = None
            
            sum_div = job.find_elements_by_class_name('summary')[0]
            
            try:
                time.sleep(sleeptime)
                if FastRun == 0:
                    sum_div.click()
                time.sleep(sleeptime) 
            except:                    
                act = ActionChains(driver)
                act.send_keys(Keys.ESCAPE).perform()
                time.sleep(sleeptime) 
                sum_div.click()
                time.sleep(sleeptime)             
            if FastRun == 0:
                try:
                    job_desc = driver.find_element_by_id('vjs-desc').text
                    j=1
                    remark_list = []
                    while RemarksValue(j,1) != 0:
                        remark_list.append(RemarksValue(j,1))
                        j += 1
                    remarks = remark_list
                except:
                    try:
                        driver.switch_to.window(driver.window_handles[1])
                        job_desc = driver.find_element_by_id('jobDescriptionText').text
                        j=1
                        remark_list = []
                        while RemarksValue(j,2) != 0:
                            remark_list.append(RemarksValue(j,2))
                            j += 1
                        remarks = remark_list                

                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                    except:
                        job_desc = ""
                        remarks = ""
                        rowfail = dataframe.shape[0]
                        fails = fails.append(({'Title':title,'Location':location,'Company':company, 'RowFail':rowfail}),ignore_index=True)
                        SaveOnFileFails(FailFileName, title, location, company,rowfail)
                        driver.switch_to.window(driver.window_handles[0])
            else:                
                job_desc = ""
                remarks = ""            
            dataframe = dataframe.append({'Title':title,'Location':location,'Company':company,'Salary':salary,
                                          'Sponsored':sponsored,'Ratings':ratings,'DatePost':DatePost,
                                          'Description':job_desc, 'Remarks':remarks},ignore_index=True)
            try:
                SaveOnFile(DataFileName, title, location, company,salary, sponsored, ratings,DatePost,job_desc, remarks)
            except:
                rowfail = dataframe.shape[0]
                fails = fails.append(({'Title':title,'Location':location,'Company':company, 'RowFail':rowfail}),ignore_index=True)                    
                SaveOnFileFails(FailFileName, title, location, company,rowfail)
    
dataframe.to_csv(DataFileName + 'canada.csv',index=False)

<ipython-input-6-ea2cf43bd825>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
<ipython-input-6-ea2cf43bd825>:48: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  all_jobs=driver.find_elements_by_class_name('result')
C:\Users\danie\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:463: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


IndexError: list index out of range